In [13]:
import time
import httpx

In [17]:
base_url = "https://swapi.dev/api/people"
urls = []
for character_id in range(1, 6):
    urls.append(f"{base_url}/{character_id}/")

In [18]:
urls

['https://swapi.dev/api/people/1/',
 'https://swapi.dev/api/people/2/',
 'https://swapi.dev/api/people/3/',
 'https://swapi.dev/api/people/4/',
 'https://swapi.dev/api/people/5/']

# Making sync requests

In [24]:
s = time.perf_counter()
for url in urls:
    r = httpx.get(url)
    print(r.json()["name"])
elapsed = time.perf_counter() - s
print(f"fetch executed in {elapsed:0.2f} seconds.")

Luke Skywalker
C-3PO
R2-D2
Darth Vader
Leia Organa
fetch executed in 0.35 seconds.


# Threads

In [11]:
import concurrent.futures

In [23]:
s = time.perf_counter()
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_url = {executor.submit(httpx.get, url): url for url in urls}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            r = future.result()
            print(r.json()["name"])
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
elapsed = time.perf_counter() - s
print(f"fetch executed in {elapsed:0.2f} seconds.")

R2-D2
Leia Organa
C-3PO
Darth Vader
Luke Skywalker
fetch executed in 0.14 seconds.


# Async

In [25]:
import asyncio

In [26]:
s = time.perf_counter()
async with httpx.AsyncClient() as client:
    responses = await asyncio.gather(*[client.get(url) for url in urls])
    for r in responses:
        print(r.json()["name"])
elapsed = time.perf_counter() - s
print(f"fetch executed in {elapsed:0.2f} seconds.")

Luke Skywalker
C-3PO
R2-D2
Darth Vader
Leia Organa
fetch executed in 0.15 seconds.
